In [46]:
from LLM_pipeline_V2 import LLMCall, ClusteringManager
import shutil
import time
import base64
from PIL import Image
import io
import os
import pandas as pd
from tabulate import tabulate
import numpy as np
from Categories_TreeStructure import create_category_folders_from_csv
import torch
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity
import json
import re
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.schema import AIMessage
from langchain_core.runnables import RunnableLambda

In [47]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [48]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [49]:
def prompt_func(data):
    type_ = data["type"]
    text = data["text"]
    content_parts = []

    if type_ == "keywords":
        image = data["image"]
        image_part = {
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{image}",
        }
        content_parts.append(image_part)

    #system_message = SystemMessage(content=data["system_message_text"])

    text_part = {"type": "text", "text": text}

    content_parts.append(text_part)

    human_message = HumanMessage(content=content_parts)

    #return [system_message, human_message]
    return [human_message]


In [50]:
def call_func(chain, prompt):
    try:
        response = chain.invoke(prompt)

        if isinstance(response, AIMessage):
            response_text = response.content
        else:
            response_text = str(response)  # Conversion en string si nécessaire

        #print(f"Reponse du llm : {response_text}")

        return extract_json(response_text)

    except Exception as e:
        print(f"Erreur de parsing JSON : {e}. Nouvelle tentative...")
        return -1

In [51]:
directory = "temp"
calling = LLMCall(directory)
df = calling.df
clustering_manager = ClusteringManager(df)
llm = ChatOllama(model="gemma3")

In [52]:
df

,image_name,path,date_time,localisation
0,IMG_20250105_144147.jpg,temp\IMG_20250105_144147.jpg,2025:01:05 14:41:47,"{1: 'N', 2: (46.0, 48.0, 43.43), 3: 'W', 4: (7..."
1,IMG_20250105_144436.jpg,temp\IMG_20250105_144436.jpg,2025:01:05 14:44:36,"{1: 'N', 2: (46.0, 48.0, 43.43), 3: 'W', 4: (7..."
2,IMG_20250105_144540.jpg,temp\IMG_20250105_144540.jpg,2025:01:05 14:45:40,"{1: 'N', 2: (46.0, 48.0, 43.31), 3: 'W', 4: (7..."
3,IMG_20250105_144750.jpg,temp\IMG_20250105_144750.jpg,2025:01:05 14:47:50,"{1: 'N', 2: (46.0, 48.0, 43.31), 3: 'W', 4: (7..."
4,IMG_20250105_145543_BURST1.jpg,temp\IMG_20250105_145543_BURST1.jpg,2025:01:05 14:55:42,"{1: 'N', 2: (46.0, 48.0, 43.17), 3: 'W', 4: (7..."
5,IMG_20250105_150144.jpg,temp\IMG_20250105_150144.jpg,2025:01:05 15:01:45,"{1: 'N', 2: (46.0, 48.0, 43.33), 3: 'W', 4: (7..."
6,IMG_20250105_161636.jpg,temp\IMG_20250105_161636.jpg,2025:01:05 16:16:36,"{1: 'N', 2: (46.0, 48.0, 48.99), 3: 'W', 4: (7..."
7,IMG_20250105_165438.jpg,temp\IMG_20250105_165438.jpg,2025:01:05 16:54:39,"{1: 'N', 2: (46.0, 48.0, 44.34), 3: 'W', 4: (7..."
8,IMG_20250105_165450.jpg,temp\IMG_20250105_165450.jpg,2025:01:05 16:54:51,"{1: 'N', 2: (46.0, 48.0, 44.58), 3: 'W', 4: (7..."
9,IMG_20250105_165457.jpg,temp\IMG_20250105_165457.jpg,2025:01:05 16:54:58,"{1: 'N', 2: (46.0, 48.0, 44.62), 3: 'W', 4: (7..."


In [53]:
cluster_df, clusters_by_day = clustering_manager.perform_neighbors_clustering(threshold=0.6, n_neighbors=3)

Début du clustering par voisins proches...
Génération des embeddings pour le jour: 2025:01:05
Génération des embeddings pour le jour: 2025:01:07
Génération des embeddings pour le jour: 2025:01:08
Génération des embeddings pour le jour: 2025:01:09
Génération des embeddings pour le jour: 2025:01:10
Génération des embeddings pour le jour: 2025:01:11
Génération des embeddings pour le jour: 2025:01:13
Génération des embeddings pour le jour: 2025:01:14
Génération des embeddings pour le jour: 2025:01:19
Génération des embeddings pour le jour: 2025:01:22
Génération des embeddings pour le jour: 2025:01:24
Clustering du jour 2025:01:05 avec 10 images...
Les photos temp\IMG_20250105_144147.jpg et temp\IMG_20250105_144436.jpg ont une similarité de 0.79
Les photos temp\IMG_20250105_144147.jpg et temp\IMG_20250105_144540.jpg ont une similarité de 0.66
Les photos temp\IMG_20250105_144147.jpg et temp\IMG_20250105_144750.jpg ont une similarité de 0.69
Les photos temp\IMG_20250105_144436.jpg et temp\IMG

In [54]:
for day, day_clusters in clusters_by_day.items():
    print(f"Day: {day}")
    for cluster_name, image_paths in day_clusters.items():
        print(f"Cluster: {cluster_name}")
        for image_path in image_paths:
            print(f"  {image_path}")

Day: 2025:01:05
Cluster: cluster_0
  temp\IMG_20250105_144147.jpg
  temp\IMG_20250105_144750.jpg
  temp\IMG_20250105_144436.jpg
  temp\IMG_20250105_145543_BURST1.jpg
  temp\IMG_20250105_144540.jpg
  temp\IMG_20250105_150144.jpg
  temp\IMG_20250105_161636.jpg
  temp\IMG_20250105_165438.jpg
  temp\IMG_20250105_165450.jpg
  temp\IMG_20250105_165457.jpg
Day: 2025:01:07
Cluster: cluster_1
  temp\IMG_20250107_180316.jpg
Day: 2025:01:08
Cluster: cluster_2
  temp\IMG_20250108_143517.jpg
Day: 2025:01:09
Cluster: cluster_3
  temp\IMG_20250109_174849.jpg
Day: 2025:01:10
Cluster: cluster_4
  temp\IMG_20250110_163528.jpg
Cluster: cluster_5
  temp\IMG_20250110_164345.jpg
Day: 2025:01:11
Cluster: cluster_6
  temp\IMG_20250111_152149.jpg
  temp\IMG_20250111_154046.jpg
  temp\IMG_20250111_152150.jpg
Day: 2025:01:13
Cluster: cluster_7
  temp\IMG_20250113_201010.jpg
Day: 2025:01:14
Cluster: cluster_8
  temp\IMG_20250114_193654.jpg
Day: 2025:01:19
Cluster: cluster_9
  temp\IMG_20250119_131124_BURST1.jpg
 

In [55]:
def get_representative_image(image_paths):
    embeddings = clustering_manager.image_embedding(paths=image_paths)
    all_scores = []
    for i in range(len(image_paths)):
        similarity = 0
        for j in range(len(image_paths)):
            similarity += np.dot(embeddings[i], embeddings[j])
        all_scores.append(similarity)
    best_score_index = all_scores.index(max(all_scores))
    return image_paths[best_score_index]

In [56]:
images_to_send = []
for day, day_clusters in clusters_by_day.items():
    print(f"Day: {day}")
    for cluster_name, image_paths in day_clusters.items():
        if len(image_paths) > 1:
            print(f"Cluster: {cluster_name}")
            representative_image = get_representative_image(image_paths)
            print(f"Most representative image: {representative_image}")
            images_to_send.append(representative_image)

Day: 2025:01:05
Cluster: cluster_0
Most representative image: temp\IMG_20250105_144436.jpg
Day: 2025:01:07
Day: 2025:01:08
Day: 2025:01:09
Day: 2025:01:10
Day: 2025:01:11
Cluster: cluster_6
Most representative image: temp\IMG_20250111_152149.jpg
Day: 2025:01:13
Day: 2025:01:14
Day: 2025:01:19
Cluster: cluster_9
Most representative image: temp\IMG_20250119_131334_BURST1.jpg
Day: 2025:01:22
Cluster: cluster_12
Most representative image: temp\IMG_20250122_155558.jpg
Day: 2025:01:24
Cluster: cluster_13
Most representative image: temp\IMG_20250124_092101.jpg


In [78]:
def llm_call(image_paths, keywords_chain):
    categories = []
    for i in range(0, len(image_paths)):
        print(f"Image {i} : {image_paths[i]}")
        image_b64 = encode_image(image_paths[i])
        image_name = os.path.basename(image_paths[i])

        wrong_json = True
        max_iter = 100
        while wrong_json and max_iter > 0:
            prompt = {
                "type": "keywords",
                "text": f"""Je fais une application pour trier mes photos et j'aimerais que tu m'aides à nommer les dossiers des photos. Comment pourrais-je résumer en un mot la photo suivante : {image_name} ? Si tu ne trouves vraiment rien, donne moi "None". Réponds-moi au format JSON : {{"nom_dossier" : "le_nom_que_tu_trouves" }}.""",
                "image": image_b64
            }

            output = call_func(keywords_chain, prompt)
            print(f"Output : {output}\n")

            if output is None:
                max_iter -= 1
                print(f"On re-essaie avec au maximum : {max_iter}\n")
            else:
                categories.append(output)
                wrong_json = False

    return categories

In [79]:
def pipeline_call(image_paths, df, llm):
    image_data = df
    new_image_paths = image_paths

    prompt_chain = RunnableLambda(prompt_func)
    keyword_chain = prompt_chain | llm

    image_data = llm_call(new_image_paths, keyword_chain)

    return image_data

In [80]:
df = pipeline_call(images_to_send, df, llm)

Image 0 : temp\IMG_20250105_144436.jpg
Output : {'nom_dosser': 'Escalier'}

Image 1 : temp\IMG_20250111_152149.jpg
Output : {'nom_dosser': 'Ski'}

Image 2 : temp\IMG_20250119_131334_BURST1.jpg
Output : {'nom_dosser': 'Neige'}

Image 3 : temp\IMG_20250122_155558.jpg
Output : {'nom_dosser': '20250122'}

Image 4 : temp\IMG_20250124_092101.jpg
Output : {'nom_dosier': 'Selfie'}

